In [1]:
import mph
import re
import numpy as np
from scipy import integrate
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
client = mph.start()
model = client.load('System_batch.mph')

System_batch
├─ parameters
│  ├─ Rate constants
│  └─ Light parametrs
├─ functions
├─ components
│  └─ Component 
├─ geometries
├─ views
├─ selections
├─ coordinates
├─ variables
├─ couplings
├─ physics
│  └─ Reaction Engineering
│     ├─ Initial Values 
│     ├─ Species: Q
│     ├─ Species: DH
│     ├─ Species: Qm
│     ├─ Species: DHp
│     ├─ Species: QH
│     ├─ Species: D
│     ├─ Species: QHD
│     ├─ Species: QHH
│     ├─ Species: prod
│     ├─ Species: QD
│     ├─ Ke: Q + DH => Qm + DHp
│     ├─ KH: Qm + DHp => QH + D
│     ├─ Kr: QH + D => QHD
│     ├─ Kp: QHD =>QHH + prod
│     ├─ KqH//Kdisp: Q + QHH <=> 2 QH
│     ├─ Kph: Q => prod
│     ├─ Ks: Q + QHD =>QH + QD
│     ├─ Kd//Kc: QD <=>Q + D
│     └─ KrD: 2 D => prod
├─ multiphysics
├─ materials
├─ meshes
├─ studies
│  └─ Study control
│     ├─ Parametric Sweep
│     └─ Time Dependent
├─ solutions
│  ├─ Solution
│  │  ├─ Compile Equations: Time Dependent
│  │  ├─ Dependent Variables
│  │  │  ├─ Concentration (comp.ODE1)
│  │ 

In [10]:
model.solve()

In [11]:
model.outer('Data')

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([0.e+00, 1.e-05, 2.e-05, 3.e-05, 4.e-05, 5.e-05, 6.e-05, 7.e-05,
        8.e-05, 9.e-05, 1.e-04]))

In [30]:
def evaluate_species(model: mph.Model, outer_number, comsol_dataset='Data'):
    reaction_node = model / 'physics' / 'Reaction Engineering'
    _reaction_node_children = [i.name() for i in reaction_node.children()]

    species = re.findall(
        string='\n'.join(_reaction_node_children),
        pattern='Species: (.*)',
    )
    species_concentration_names = ['reaction.c_' + specie for specie in species]
    row_data = model.evaluate(
        ['t'] + species_concentration_names,
        dataset=comsol_dataset,
        outer=outer_number,
    )
    return pd.DataFrame(row_data, columns=['Time'] + species)


In [34]:
df = evaluate_species(model,3)

In [33]:
from structure import Const, Solve, db
from tqdm import tqdm as _tqdm

In [27]:
params={key: float(value) for key, value in model.parameters().items() if 'light' not in key}


In [28]:
params['light']=1.e-05

In [35]:
df

,Time,Q,DH,Qm,DHp,QH,D,QHD,QHH,prod,QD
0,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,7.812500e-10,0.999984,0.999984,0.000016,0.000016,1.907229e-10,1.907229e-10,0.000000e+00,0.000000e+00,7.812378e-15,1.490023e-19
2,1.369135e-09,0.999973,0.999973,0.000027,0.000027,6.314697e-10,6.314697e-10,2.344216e-19,2.344213e-19,1.369130e-14,5.202238e-19
3,2.544905e-09,0.999949,0.999949,0.000051,0.000051,3.676579e-09,3.676579e-09,1.612758e-17,1.612754e-17,2.546430e-14,4.842815e-18
4,4.896446e-09,0.999902,0.999902,0.000098,0.000098,2.621009e-08,2.621009e-08,1.631562e-15,1.631554e-15,5.059284e-14,6.647087e-17
...,...,...,...,...,...,...,...,...,...,...,...
358,1.932690e-04,0.402270,0.114060,0.000960,0.000960,7.122499e-04,5.249525e-04,3.077587e-01,2.882545e-01,2.883263e-01,4.382112e-05
359,1.954724e-04,0.400869,0.112059,0.000950,0.000950,7.075737e-04,5.189048e-04,3.085747e-01,2.888545e-01,2.889266e-01,4.436693e-05
360,1.976759e-04,0.399495,0.110099,0.000940,0.000940,7.029892e-04,5.129521e-04,3.093760e-01,2.894407e-01,2.895133e-01,4.491228e-05
361,1.998794e-04,0.398149,0.108181,0.000931,0.000931,6.984939e-04,5.070920e-04,3.101631e-01,2.900136e-01,2.900866e-01,4.545717e-05


In [52]:
# FIXME: atomise querries
def solution_to_sql(
    df: pd.DataFrame,
    name,
    desc,
    params: dict,
):
    with db:
        solve = Solve.create(
            name=name,
            desc=desc,
            data=df.to_json(index=True),
        )

        consts = [{'solve': solve,'name': key,'value': value}
        for key,value in params.items()] #yapf: disable

        Const.insert_many(consts).execute()


In [53]:
def check(string):
    if string is None:
        while string != 'q':
            string = input(f'Set {string=}, to quit - q:')
            string = string.strip()
    return string

In [54]:
def change_params(model:mph.Model,changed_params:dict):
    for key,value in changed_params.items():
        model.parameter(name=key,value=value)
    return model.parameters()

In [55]:
name = 1
desc = 2

# TODO: logs
def sweep_to_sql(
    model: mph.Model,
    name,
    desc=None,
):
    _, light_sweep = model.outer('Data')
    i = 1
    params = {
            key: float(value)
            for key,value in
            model.parameters().items()
            if 'light' not in key
        }
    for light_value in light_sweep:

         #yapf: disable
        params['light'] = light_value

        df = evaluate_species(model, i)

        solution_to_sql(
            df=df,
            name=name,
            desc=desc,
            params=params,
        )

        i += 1


In [56]:
sweep_to_sql(model=model,name='test_sweep')